In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

data_a = pd.read_csv("dataset/dataset-a.csv")
data_b = pd.read_csv("dataset/dataset-b.csv")
data_c = pd.read_csv("dataset/dataset-c.csv")
data_d = pd.read_csv("dataset/dataset-d.csv")


In [4]:
data_a.head()

,y,x
0,-0.323298,0.548814
1,0.366986,0.715189
2,0.074002,0.602763
3,-0.199561,0.544883
4,-0.403890,0.423655
